In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix,mean_absolute_error,mean_squared_error
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor



In [2]:
train_data=pd.read_csv("/kaggle/input/predict-the-age-of-a-fossil/train_data.csv")

In [3]:
train_data.head()

,uranium_lead_ratio,carbon_14_ratio,radioactive_decay_series,stratigraphic_layer_depth,geological_period,paleomagnetic_data,inclusion_of_other_fossils,isotopic_composition,surrounding_rock_type,stratigraphic_position,fossil_size,fossil_weight,age
0,0.738061,0.487707,0.907884,91.17,Cretaceous,Normal polarity,False,0.915951,Conglomerate,Middle,50.65,432.00,43523
1,0.560096,0.341738,1.121302,165.44,Cambrian,Normal polarity,False,0.803968,Limestone,Top,48.85,353.29,44112
2,0.424773,0.218493,0.103855,218.98,Cambrian,Normal polarity,True,0.792441,Shale,Bottom,37.66,371.33,43480
3,0.349958,0.704649,0.383617,51.09,Permian,Normal polarity,True,0.074636,Limestone,Bottom,39.10,232.84,30228
4,0.886811,0.777494,0.593254,313.72,Devonian,Normal polarity,True,1.646640,Shale,Top,90.84,277.67,67217


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   uranium_lead_ratio          4398 non-null   float64
 1   carbon_14_ratio             4398 non-null   float64
 2   radioactive_decay_series    4398 non-null   float64
 3   stratigraphic_layer_depth   4398 non-null   float64
 4   geological_period           4398 non-null   object 
 5   paleomagnetic_data          4398 non-null   object 
 6   inclusion_of_other_fossils  4398 non-null   bool   
 7   isotopic_composition        4398 non-null   float64
 8   surrounding_rock_type       4398 non-null   object 
 9   stratigraphic_position      4398 non-null   object 
 10  fossil_size                 4398 non-null   float64
 11  fossil_weight               4398 non-null   float64
 12  age                         4398 non-null   int64  
dtypes: bool(1), float64(7), int64(1),

In [5]:
train_data.describe()

,uranium_lead_ratio,carbon_14_ratio,radioactive_decay_series,stratigraphic_layer_depth,isotopic_composition,fossil_size,fossil_weight,age
count,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000,4398.000000
mean,0.486938,0.430116,0.532054,152.832801,0.938315,70.702460,326.655177,40586.656435
std,0.257132,0.233536,0.287929,86.272234,0.521643,37.352584,187.032333,15200.700905
min,0.000241,0.000244,0.000076,0.130000,0.000275,0.130000,0.620000,4208.000000
25%,0.294091,0.246269,0.316775,85.390000,0.533519,42.487500,181.047500,29766.250000
50%,0.473715,0.416046,0.511830,146.000000,0.905684,68.785000,308.260000,39567.000000
75%,0.658012,0.590752,0.723805,211.932500,1.289307,96.440000,455.520000,50550.500000
max,1.533270,1.000000,1.513325,494.200000,3.071434,216.390000,1010.090000,103079.000000


In [6]:
train_data.duplicated().sum()

0

In [7]:
test_data=pd.read_csv("/kaggle/input/predict-the-age-of-a-fossil/test_data.csv")

In [8]:
test_data.duplicated().sum()

0

In [9]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   uranium_lead_ratio          1100 non-null   float64
 1   carbon_14_ratio             1100 non-null   float64
 2   radioactive_decay_series    1100 non-null   float64
 3   stratigraphic_layer_depth   1100 non-null   float64
 4   geological_period           1100 non-null   object 
 5   paleomagnetic_data          1100 non-null   object 
 6   inclusion_of_other_fossils  1100 non-null   bool   
 7   isotopic_composition        1100 non-null   float64
 8   surrounding_rock_type       1100 non-null   object 
 9   stratigraphic_position      1100 non-null   object 
 10  fossil_size                 1100 non-null   float64
 11  fossil_weight               1100 non-null   float64
 12  age                         1100 non-null   int64  
dtypes: bool(1), float64(7), int64(1),

In [10]:
wrapper={False:0.0,True:1.0}
train_data.inclusion_of_other_fossils.replace(wrapper,inplace=True)

/tmp/ipykernel_33/4040812128.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data.inclusion_of_other_fossils.replace(wrapper,inplace=True)
/tmp/ipykernel_33/4040812128.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data.inclusion_of_other_fossils.replace(wrapper,inplace=True)


In [11]:
wrapper={False:0.0,True:1.0}
test_data.inclusion_of_other_fossils.replace(wrapper,inplace=True)

/tmp/ipykernel_33/2595131768.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data.inclusion_of_other_fossils.replace(wrapper,inplace=True)
/tmp/ipykernel_33/2595131768.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data.inclusion_of_other_fossils.replace(wrapper,inplace=True)


In [12]:
y_train=train_data['age']
y_test=test_data['age']

In [13]:
train_data.drop('age',axis=1,inplace=True)
test_data.drop('age',axis=1,inplace=True)

In [14]:
one_encoder =OneHotEncoder()
stratigraphic_position_encoded =one_encoder.fit_transform(train_data[['stratigraphic_position']]).toarray()
feature_names3 = one_encoder.get_feature_names_out(['stratigraphic_position'])


In [15]:
SP_encoded =pd.DataFrame(stratigraphic_position_encoded,columns=feature_names3)

In [16]:

surrounding_rock_encoded =one_encoder.fit_transform(train_data[['surrounding_rock_type']]).toarray()
feature_names2 = one_encoder.get_feature_names_out(['surrounding_rock_type'])


In [17]:
S_encoded =pd.DataFrame(surrounding_rock_encoded,columns=feature_names2)

In [18]:

paleomagnetic_encoded =one_encoder.fit_transform(train_data[['paleomagnetic_data']]).toarray()
feature_names1 = one_encoder.get_feature_names_out(['paleomagnetic_data'])


In [19]:
P_encoded =pd.DataFrame(paleomagnetic_encoded,columns=feature_names1)

In [20]:

geo_encoded =one_encoder.fit_transform(train_data[['geological_period']]).toarray()
feature_names = one_encoder.get_feature_names_out(['geological_period'])


In [21]:
G_encoded =pd.DataFrame(geo_encoded,columns=feature_names)

In [22]:
G_encoded.head()

,geological_period_Cambrian,geological_period_Carboniferous,geological_period_Cretaceous,geological_period_Devonian,geological_period_Jurassic,geological_period_Neogene,geological_period_Ordovician,geological_period_Paleogene,geological_period_Permian,geological_period_Silurian,geological_period_Triassic
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
x_train=pd.concat([train_data,G_encoded,P_encoded,S_encoded,SP_encoded],axis=1)

In [24]:
x_train.drop(['paleomagnetic_data',"surrounding_rock_type","geological_period","stratigraphic_position"],inplace=True,axis=1)

In [25]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 28 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   uranium_lead_ratio                    4398 non-null   float64
 1   carbon_14_ratio                       4398 non-null   float64
 2   radioactive_decay_series              4398 non-null   float64
 3   stratigraphic_layer_depth             4398 non-null   float64
 4   inclusion_of_other_fossils            4398 non-null   float64
 5   isotopic_composition                  4398 non-null   float64
 6   fossil_size                           4398 non-null   float64
 7   fossil_weight                         4398 non-null   float64
 8   geological_period_Cambrian            4398 non-null   float64
 9   geological_period_Carboniferous       4398 non-null   float64
 10  geological_period_Cretaceous          4398 non-null   float64
 11  geological_period

In [26]:
x_train.head()

,uranium_lead_ratio,carbon_14_ratio,radioactive_decay_series,stratigraphic_layer_depth,inclusion_of_other_fossils,isotopic_composition,fossil_size,fossil_weight,geological_period_Cambrian,geological_period_Carboniferous,...,geological_period_Triassic,paleomagnetic_data_Normal polarity,paleomagnetic_data_Reversed polarity,surrounding_rock_type_Conglomerate,surrounding_rock_type_Limestone,surrounding_rock_type_Sandstone,surrounding_rock_type_Shale,stratigraphic_position_Bottom,stratigraphic_position_Middle,stratigraphic_position_Top
0,0.738061,0.487707,0.907884,91.17,0.0,0.915951,50.65,432.00,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.560096,0.341738,1.121302,165.44,0.0,0.803968,48.85,353.29,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.424773,0.218493,0.103855,218.98,1.0,0.792441,37.66,371.33,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.349958,0.704649,0.383617,51.09,1.0,0.074636,39.10,232.84,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.886811,0.777494,0.593254,313.72,1.0,1.646640,90.84,277.67,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [27]:
one_encoder =OneHotEncoder()
stratigraphic_position_encoded_test =one_encoder.fit_transform(test_data[['stratigraphic_position']]).toarray()
feature_name3_test = one_encoder.get_feature_names_out(['stratigraphic_position'])


In [28]:
SP_encoded_test =pd.DataFrame(stratigraphic_position_encoded_test,columns=feature_name3_test)

In [29]:

surrounding_rock_encoded_test =one_encoder.fit_transform(test_data[['surrounding_rock_type']]).toarray()
feature_names2_test = one_encoder.get_feature_names_out(['surrounding_rock_type'])


In [30]:
S_encoded_test =pd.DataFrame(surrounding_rock_encoded_test,columns=feature_names2_test)

In [31]:

paleomagnetic_encoded_test =one_encoder.fit_transform(test_data[['paleomagnetic_data']]).toarray()
feature_names1_test = one_encoder.get_feature_names_out(['paleomagnetic_data'])


In [32]:
P_encoded_test =pd.DataFrame(paleomagnetic_encoded_test,columns=feature_names1_test)

In [33]:

geo_encoded_test =one_encoder.fit_transform(test_data[['geological_period']]).toarray()
feature_names_test = one_encoder.get_feature_names_out(['geological_period'])


In [34]:
G_encoded_test =pd.DataFrame(geo_encoded_test,columns=feature_names_test)

In [35]:
x_test=pd.concat([test_data,G_encoded_test,P_encoded_test,S_encoded_test,SP_encoded_test],axis=1)

In [36]:
x_test.drop(['paleomagnetic_data',"surrounding_rock_type","geological_period","stratigraphic_position"],inplace=True,axis=1)

In [37]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 28 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   uranium_lead_ratio                    1100 non-null   float64
 1   carbon_14_ratio                       1100 non-null   float64
 2   radioactive_decay_series              1100 non-null   float64
 3   stratigraphic_layer_depth             1100 non-null   float64
 4   inclusion_of_other_fossils            1100 non-null   float64
 5   isotopic_composition                  1100 non-null   float64
 6   fossil_size                           1100 non-null   float64
 7   fossil_weight                         1100 non-null   float64
 8   geological_period_Cambrian            1100 non-null   float64
 9   geological_period_Carboniferous       1100 non-null   float64
 10  geological_period_Cretaceous          1100 non-null   float64
 11  geological_period

In [38]:
x_test.head()

,uranium_lead_ratio,carbon_14_ratio,radioactive_decay_series,stratigraphic_layer_depth,inclusion_of_other_fossils,isotopic_composition,fossil_size,fossil_weight,geological_period_Cambrian,geological_period_Carboniferous,...,geological_period_Triassic,paleomagnetic_data_Normal polarity,paleomagnetic_data_Reversed polarity,surrounding_rock_type_Conglomerate,surrounding_rock_type_Limestone,surrounding_rock_type_Sandstone,surrounding_rock_type_Shale,stratigraphic_position_Bottom,stratigraphic_position_Middle,stratigraphic_position_Top
0,0.469986,1.000000,0.667595,29.58,0.0,0.583560,120.12,73.83,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.619865,0.474208,1.218381,69.87,1.0,0.942719,72.82,191.68,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.767736,0.478731,0.119801,96.38,0.0,0.377531,105.47,82.25,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.275121,0.400594,0.634760,134.10,1.0,0.323820,94.99,47.99,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.407470,0.039705,0.824597,124.10,0.0,1.219120,139.93,532.62,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [39]:
y_test.shape

(1100,)

In [40]:
ln_model=LinearRegression().fit(x_train,y_train)

y_pred =ln_model.predict(x_test)


In [41]:
mae=mean_absolute_error(y_pred,y_test)
mse=mean_squared_error(y_pred,y_test)
print(f"Mean absolute Error using mean absolute error: {mae}")
print(f"Mean squared Error: {mse}")

Mean absolute Error using mean absolute error: 1426.578880003005
Mean squared Error: 3453169.726078272


In [42]:
knn =KNeighborsRegressor(n_neighbors=7)
knn.fit(x_train,y_train)


KNeighborsRegressor(n_neighbors=7)

In [43]:
y_pred =knn.predict(x_test)

In [44]:
mae=mean_absolute_error(y_pred,y_test)
mse=mean_squared_error(y_pred,y_test)
print(f"Mean absolute Error using knn: {mae}")
print(f"Mean squared Error using knn: {mse}")

Mean absolute Error using knn: 12509.683506493506
Mean squared Error using knn: 243196827.11719856


In [45]:
RFR= RandomForestRegressor(n_estimators=100, max_depth =4,random_state=42)
RFR.fit(x_train,y_train)

y_pred =RFR.predict(x_test)

In [46]:
mae=mean_absolute_error(y_pred,y_test)
mse=mean_squared_error(y_pred,y_test)
print(f"Mean absolute Error using RFR: {mae}")
print(f"Mean squared Error using RFR: {mse}")

Mean absolute Error using RFR: 4067.33767039245
Mean squared Error using RFR: 26591695.870810136
